In [1]:
import cv2
import numpy as np
import time

In [2]:
"""
Reads image and returns the image and its gray scale format. 
"""
def read_img(img_path:str):
    img= cv2.imread(img_path)
    return img,cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [3]:
def show_image(window_names:list, images:list):
    for name,img in zip(window_names,images):
        cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Section1

### Line Detection

<p> We will cover 3 different techniques wich are :</p>
<ul>
<li>Hough Lines
<li>Hough Lines Transform
<li>Fast Line Detector
</ul>


In [4]:
coral_img,coral_gray_img= read_img('pics/coralReef.png')
show_image(['Coral REEF', 'Gray Coral REEF'],[ coral_img, coral_gray_img])
# show_image(, coral_gray_img)

In [5]:
sharpen_kernel= (1/12)*np.array([
   [ -1,-2,-1]
   ,[-2,24,-2],
   [-1,-2,-1]])
result= coral_gray_img
## First let's do some preprocessing
blurring= [3,3,5,5]
for i in blurring:
    # result= cv2.filter2D(result,-1,sharpen_kernel)
    result= cv2.medianBlur(result ,i)
    result= cv2.filter2D(result,-1,sharpen_kernel)
# result= cv2.medianBlur(result ,5)

edges= cv2.Canny(result,25,255)
show_image(['Gray Coral REEF','Result', 'EDGE DETECTION'],[  coral_gray_img,result,edges])



In [6]:
from math import pi,cos,sin, floor
currentTime= time.time()
vertical_lines= cv2.HoughLines(edges,1,pi/180,65,min_theta=0,max_theta=pi*10/180)
horizontal_lines= cv2.HoughLines(edges,1,pi/180,65,min_theta=pi*80/180,max_theta=pi*90/180)
print(f'Time Elapsed is {(time.time()-currentTime)*1000} ms')
output_image= coral_img.copy()
def drawLines(lines,image):
    for line in lines:
        line=line[0]
        rho = line[0]
        theta = line [1];
        a = cos(theta)
        b = sin(theta);
        x0 = a*rho
        y0 = b*rho;
        x1 = floor(x0 + 1000*(-b))
        y1 =floor(y0 + 1000*(a))
        x2 =floor(x0 - 1000*(-b))
        y2 = floor(y0 - 1000*(a))
        cv2.line( image,(x1,y1),(x2,y2),(0,0,255), 3, cv2.LINE_AA);
drawLines(vertical_lines,output_image)
drawLines(horizontal_lines,output_image)

show_image(['Gray Image','Edges Image', 'Line Detection Image'], [coral_gray_img,edges,output_image])
    

Time Elapsed is 2.8395652770996094 ms


In [8]:
currentTime= time.time()
lines= cv2.HoughLinesP(edges,1,pi/180,65, minLineLength= 0, maxLineGap=0)
print(f'Time Elapsed is {(time.time()-currentTime)*1000} ms')
output_img= coral_img.copy()
for line in lines:
    line=line[0]
    cv2.line(output_img, (line[0],line[1]),(line[2],line[3]),(0,0,255), 3, cv2.LINE_AA);
show_image(['Probabilistic Detection'], [output_image])

    

Time Elapsed is 5.005359649658203 ms


In [9]:
# Create default Fast Line Detector class
fld = cv2.ximgproc.createFastLineDetector()
# Get line vectors from the image
lines = fld.detect(edges)
lines_without_blur = fld.detect(coral_gray_img)

# # Draw lines on the image
line_on_image = fld.drawSegments(coral_img, lines)
line_on_image2 = fld.drawSegments(coral_img, lines_without_blur)

show_image(['Line Detector  On Processed', 'Line detector without Porcessing'], [line_on_image, line_on_image2])

# # Plot
# plt.imshow(line_on_image, interpolation='nearest', aspect='auto')
# plt.show()
# return line_on_image

# Section2

In [30]:
circle_image,circle_gray_image= read_img('pics/circles.png')
show_image(['pic'], [circle_image])

In [42]:
circles= cv2.HoughCircles(circle_gray_image,cv2.HOUGH_GRADIENT,1.5,20,200,100)[0]
output_img= circle_image.copy()
for circle in circles:
    circle= circle.astype(int)
    print(circle)
    cv2.circle(output_image,(circle[0],circle[1]), circle[2], (0,255,0),-1,0,0)
    cv2.circle( output_img, (circle[0],circle[1]), circle[2], (0,0,255), 3, 8, 0 ) #Outline
show_image(['Original','HoughCircles'], [circle_image,output_img])

[51 47 33]
[219 131  33]
[134  47  29]
[ 51 131  28]
[222  47  33]
[134 132  29]


# Section 3

In [44]:
show_image(['edges'],[edges])

In [53]:
contours, hierarchy = cv2.findContours(edges,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# output_img= coral_img.copy()
output_img1= np.zeros_like(coral_img,dtype=np.uint8)
cv2.drawContours(output_img1, contours, -1, (0, 255, 0), 3)
  
contours, hierarchy = cv2.findContours(edges,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# output_img= coral_img.copy()
output_img2= np.zeros_like(coral_img,dtype=np.uint8)
cv2.drawContours(output_img2, contours, -1, (0, 255, 0), 3)

show_image(['edges','original','Contours Simple','Contours None'], [edges,coral_img,output_img1,output_img2])

In [54]:
## You can calculate area of contour.
## You can the perimeter.
## You can an approximate contour of a polygon but take care of parameters.
## See more examples here:https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html 